# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import time

import numpy as np
import torch
from bnpm import file_helpers, optimization
import sklearn.utils.class_weight
from torch import nn, optim
from tqdm import tqdm
import sklearn.linear_model
import multiprocessing as mp

import roicat.classification.classifier_util as cu
import scipy.sparse
import roicat
import bnpm.h5_handling
from pathlib import Path
import shutil
import warnings
import umap

import matplotlib.pyplot as plt
import matplotlib.offsetbox
import json
import os
import natsort
from roicat import helpers
# from kymatio.torch import Scattering2D
import gc
import functools

2023-06-21 16:00:58.652563: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Specify Initial Parameters

In [2]:
path_params = None # Path(r"")
# directory_data = r'/Users/josh/analysis/outputs/ROICaT/classification/00_data_ingestion'
directory_data = r'/Users/josh/analysis/outputs/ROICaT/classification/labeling_interactive/feature_label_combo.npy'
directory_save = r'/Users/josh/analysis/outputs/ROICaT/classification/02_classifier_train'
testing = True
save_ROIs = True
save_latents = True

In [3]:
directory_save = Path(directory_save)
directory_save.mkdir(exist_ok=True, parents=True)

params = file_helpers.json_load(str(Path(path_params).resolve())) if path_params is not None else None

# Data Importing — Suite2p

In [4]:
dir_allOuterFolders = str(Path('/Users/josh/analysis/data/ROICaT/classification/stat_s2p').resolve())

pathSuffixToStat = 'stat.npy'
pathSuffixToOps = 'ops.npy'
pathSuffixToLabels = 'labels_round2_sesh2.npy'

paths_allStat = helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToStat,
    depth=4,
)
paths_allOps = helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToOps,
    depth=4,
)
paths_allLabels = helpers.find_paths(
    dir_outer=dir_allOuterFolders,
    reMatch=pathSuffixToLabels,
    depth=4,
)

display(paths_allStat)
display(paths_allOps)

#Import data
data = roicat.data_importing.Data_suite2p(
    paths_statFiles=paths_allStat,
    paths_opsFiles=paths_allOps,
    class_labels=[np.load(path_label, allow_pickle=True) for path_label in paths_allLabels],
    um_per_pixel=2.0,
    new_or_old_suite2p='new',
    out_height_width=[36, 36],
    type_meanImg='meanImgE',
    verbose=True,
);

['/Users/josh/analysis/data/ROICaT/classification/stat_s2p/stat.npy']

['/Users/josh/analysis/data/ROICaT/classification/stat_s2p/ops.npy']

/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:191: UserWarning: RH WARNING: unable to get cpu info. Got error: No module named 'cpuinfo'
  warnings.warn(f'RH WARNING: unable to get cpu info. Got error: {e}')
/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:210: UserWarning: RH WARNING: unable to get gpu info. Got error: No module named 'GPUtil'
  warnings.warn(f'RH WARNING: unable to get gpu info. Got error: {e}')


Starting: Importing FOV images from ops files
Completed: Set FOV_height and FOV_width successfully.
Completed: Imported 1 FOV images.
Completed: Set FOV_images for 1 sessions successfully.
Importing spatial footprints from stat files.


100%|██████████| 1/1 [00:01<00:00,  1.93s/it]


Imported 1 sessions of spatial footprints into sparse arrays.
Completed: Set spatialFootprints for 1 sessions successfully.
Completed: Created session_bool.
Completed: Created centroids.
Staring: Creating centered ROI images from spatial footprints...
Completed: Created ROI images.
Starting: Importing class labels
Labels and ROI Images match in shapes: Class labels and ROI images have the same number of sessions and the same number of ROIs in each session.
Completed: Imported labels for 1 sessions. Each session has [4898] class labels. Total number of class labels is 4898.


# Data Importing — Hand Labeled Inputs

In [14]:
filepath_labellingInteractive = str(Path('/Users/josh/analysis/outputs/ROICaT/classification/labeling_interactive/test.ROICaT.labeling.results.pkl').resolve())
labelingInteractive = roicat.helpers.pickle_load(filepath_labellingInteractive)
category_mappings, codes_categories = np.unique(labelingInteractive['labels']['label'], return_inverse=True)

In [15]:
# Import data
data = roicat.data_importing.Data_roicat();
data.set_ROI_images([labelingInteractive['images'][labelingInteractive['labels']['index']]]);
data.set_class_labels([labelingInteractive['labels']['label']]);

RH WARNING: No um_per_pixel provided. We recommend making an educated guess. Assuming 1.0 um per pixel. This will affect the embedding results.
Starting: Importing ROI images
Completed: Imported 1 sessions. Each session has [502] ROIs. Total number of ROIs is 502. The um_per_pixel is 1.0 um per pixel.
Starting: Importing class labels
Labels and ROI Images match in shapes: Class labels and ROI images have the same number of sessions and the same number of ROIs in each session.
Completed: Imported labels for 1 sessions. Each session has [502] class labels. Total number of class labels is 502.


/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:191: UserWarning: RH WARNING: unable to get cpu info. Got error: No module named 'cpuinfo'
  warnings.warn(f'RH WARNING: unable to get cpu info. Got error: {e}')
/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:210: UserWarning: RH WARNING: unable to get gpu info. Got error: No module named 'GPUtil'
  warnings.warn(f'RH WARNING: unable to get gpu info. Got error: {e}')


# Pass Through Network

In [16]:
# Neural network embedding distances
roinet = roicat.ROInet.ROInet_embedder(
    device=roicat.util.helpers.set_device('cuda:0'),
    dir_networkFiles=r"/Users/josh/analysis/models",
    download_method="check_local_first",
    download_url="https://osf.io/xwzhp/download",
    download_hash="134b170242141c26b0adbd9e0fd80d0e",
    forward_pass_version="head",
    verbose=True,
)

roinet.generate_dataloader(
    ROI_images=data.ROI_images,
    um_per_pixel=data.um_per_pixel,
    pref_plot=False,
    batchSize_dataloader=8,
    pinMemory_dataloader=True,
    numWorkers_dataloader=mp.cpu_count(),
    persistentWorkers_dataloader=True,
    prefetchFactor_dataloader=2,    
);

# roicat.visualization.display_toggle_image_stack(roinet.ROI_images_rs)

roinet.generate_latents();

gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

devices available: []
no GPU available. Using CPU.
File already exists locally: /Users/josh/analysis/models/ROInet.zip


/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:191: UserWarning: RH WARNING: unable to get cpu info. Got error: No module named 'cpuinfo'
  warnings.warn(f'RH WARNING: unable to get cpu info. Got error: {e}')
/Users/josh/analysis/github_repos/ROICaT/roicat/util.py:210: UserWarning: RH WARNING: unable to get gpu info. Got error: No module named 'GPUtil'
  warnings.warn(f'RH WARNING: unable to get gpu info. Got error: {e}')


Hash of local file matches provided hash_hex.
Extracting /Users/josh/analysis/models/ROInet.zip to /Users/josh/analysis/models.
Completed zip extraction.
['/Users/josh/analysis/models/ROInet_classification_20220902', '/Users/josh/analysis/models/ROInet_classification_20220902/ConvNext_tiny__1_0_best__simCLR_wPCA.pth', '/Users/josh/analysis/models/ROInet_classification_20220902/model.py', '/Users/josh/analysis/models/ROInet_classification_20220902/classifier.pkl', '/Users/josh/analysis/models/ROInet_classification_20220902/params.json', '/Users/josh/analysis/models/ROInet_classification_20220902/__pycache__', '/Users/josh/analysis/models/ROInet_classification_20220902/__pycache__/model.cpython-39.pyc']
Imported model from /Users/josh/analysis/models/ROInet_classification_20220902/model.py
Loaded params_model from /Users/josh/analysis/models/ROInet_classification_20220902/params.json


/Users/josh/opt/anaconda3/envs/ROICaT/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/josh/opt/anaconda3/envs/ROICaT/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Generated network using params_model
Loaded state_dict into network from /Users/josh/analysis/models/ROInet_classification_20220902/ConvNext_tiny__1_0_best__simCLR_wPCA.pth
Loaded network onto device cpu
Starting: resizing ROIs
Completed: resizing ROIs
Defined image transformations: Sequential(
  (0): ScaleDynamicRange(scaler_bounds=(0, 1))
  (1): Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
  (2): TileChannels(dim=0)
)
Defined dataset
Defined dataloader
starting: running data through network


100%|██████████| 63/63 [01:19<00:00,  1.27s/it]


completed: running data through network


In [17]:
# feature_label_combo = np.load(directory_data, allow_pickle=True)[()]
# # TODO: Add alternative data importing method for raw data when feature_label_combo does not exist

# Train / Validation / Test Split Data, Hyperparameter Tune on Validation Set, and Fit Model

In [22]:
INTEGER_MAX = np.iinfo(np.int64(0).dtype).max

# TODO: JZ, IMPLEMENT AS LOOP FOR OPTUNA FOR HYPERPARAMETER TUNING
print('Splitting data...')
# Create data splitting object for stratified sampling into train and test sets (as well as downsampling)
data_splitter = cu.Datasplit(
    features=roinet.latents,
    labels=np.concatenate(data.class_labels, axis=0),
    n_train=INTEGER_MAX,
    val_size=0.2,
    test_size=0.2,
)

print('Calculating class weights...')
num_classes = len(np.unique(data.class_labels))
class_weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced',
                                                                classes=np.unique(np.concatenate(data.class_labels, axis=0)),
                                                                y=np.concatenate(data.class_labels, axis=0))

print(f'Fitting model to data of dimensions: X: {data_splitter.features_train}, y: {data_splitter.labels_train}...')
# Create lenet model, associated optimizer, loss function, and training tracker
model = sklearn.linear_model.LogisticRegression(
   solver='lbfgs',
   fit_intercept=True,
   max_iter=10000,
   C=1e5,
   class_weight={iClassWeight:classWeight for iClassWeight, classWeight in enumerate(class_weights)},
#    class_weight=class_weights,
)
model.fit(data_splitter.features_train, data_splitter.labels_train)

print(f'Calculating tracker outputs and saving to {directory_save}...')
training_tracker = cu.TrainingTracker(
    directory_save=directory_save,
    class_weights=class_weights, # Class Weights
    n_train_actual=data_splitter.n_train_actual,
    model=({'coef':model.coef_, 'intercept':model.intercept_})
)

y_train_preds = model.predict(data_splitter.features_train).astype(int)
y_train_true = data_splitter.labels_train
y_val_preds = model.predict(data_splitter.features_val).astype(int)
y_val_true = data_splitter.labels_val

# Save training loop results from current epoch for training set
training_tracker.add_accuracy(0, 'accuracy_training', y_train_true, y_train_preds) # Generating training loss
training_tracker.add_confusion_matrix(0, 'confusionMatrix_training', y_train_true, y_train_preds) # Generating confusion matrix

# Save training loop results from current epoch for validation set
training_tracker.add_accuracy(0, 'accuracy_val', y_val_true, y_val_preds) # Generating validation accuracy
training_tracker.add_confusion_matrix(0, 'confusionMatrix_val', y_val_true, y_val_preds) # Generating validation confusion matrix

training_tracker.save_results() # TODO: JZ, ADJUST RESULTS SAVING TO SAVE CONFUSION MATRICES AS NOT A DATAFRAME CSV
training_tracker.print_results()

model_save = {
    'intercept_': model.intercept_,
    'coef_': model.coef_,
    'classes_': model.classes_,
}

Splitting data...
Calculating class weights...
Fitting model to data of dimensions: X: tensor([[-5253.1489,  1064.1238, -2189.9517,  ...,  -178.1226,    95.4517,
          -136.5313],
        [-3733.0259, -2631.0325,  1033.2117,  ...,  -199.3206,    66.0951,
           -97.6897],
        [-3840.5396,  1196.2999, -3921.7903,  ...,    39.2984,    50.9117,
          -106.1018],
        ...,
        [-4230.0610,  1560.7756, -3991.6963,  ...,    35.7600,    67.7590,
           -83.6938],
        [-5258.3848,  1584.0309, -1874.1542,  ...,    71.1284,    44.4079,
           -41.8832],
        [-4347.1313,   760.6380,  -334.8129,  ...,  -176.3295,   131.9691,
          -105.4062]]), y: [2 0 3 2 3 0 0 0 1 1 0 1 0 2 1 2 3 2 2 0 1 0 2 0 0 2 1 0 2 2 1 0 0 2 0 1 0
 0 1 3 2 3 2 3 1 2 0 2 0 3 1 2 2 1 0 1 2 1 0 1 1 2 1 3 3 1 0 1 1 2 0 0 1 1
 0 1 1 1 2 3 2 1 3 1 0 0 0 1 2 1 0 2 1 1 0 0 2 0 0 0 1 2 0 1 1 0 2 2 3 0 2
 3 2 2 2 0 3 2 0 1 0 2 2 1 1 0 2 3 2 0 2 0 1 2 2 1 1 1 0 0 1 1 1 3 1 1 0 3
 1 0 0 2 2 2 

/Users/josh/opt/anaconda3/envs/ROICaT/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Save Outputs

In [ ]:
np.save(str((Path(directory_save) / 'model.npy').resolve()), model_save, allow_pickle=True)

print(f'Saved model fit results.')